In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler


## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
heart = heart.dropna()

In [5]:
## defining target and input variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## splitting the data into 80% training and 20% testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [6]:
## scaling the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [10]:
## builiding model (logistic regression, RF)
logit_md = LogisticRegression().fit(X_train, Y_train)
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

## making predictions on the testing dataset (choosing second column)
logit_pred = logit_md.predict_proba(X_test)[:, 1]
rf_pred = rf_md.predict_proba(X_test)[:, 1]

## changing likelihoods to labes, setting a 10% cutoff
logit_labels = np.where(logit_pred < 0.1, 0, 1)
rf_labels = np.where(rf_pred < 0.1, 0, 1)

## computing recall score
logit_recall = recall_score(Y_test, logit_labels)
rf_recall = recall_score(Y_test, rf_labels)

## printing results
print('The recall score for the Logistic Regression Model is', logit_recall)
print('The recall score for the Random Forest Model is', rf_recall)

The recall score for the Logistic Regression Model is 0.8166666666666667
The recall score for the Random Forest Model is 0.8666666666666667


In [ ]:
## why do we use predict_proba? .predict = gives labels with 50% threshold
## pre

In [ ]:
## what does [;, 1] mean? [range from 0-max, column 1]

From my above results, the Random Forest model has a higher recall score, which means the that model
is preferred to predict the risk of coronary heart disease.